In [1]:
import sys
sys.path.append('../')
import pickle
from sinergym_wrappers.sinergym_wrapper import SinergymWrapper
from sinergym_wrappers.epw_scraper.epw_data import EPW_Data
from core.utils import *
import numpy as np
import pandas as pd
from itertools import starmap
from multiprocessing import Pool

weather_var_names = ['drybulb', 'relhum',
                        "winddir", "dirnorrad", "difhorrad"]
weather_var_rev_names = ["windspd"]

epw_data = EPW_Data.load("sinergym_wrappers/epw_scraper/US_epw_OU_data.pkl")
# We only need to include the default evaluation variability since we'll sample the rest later
weather_var_config = get_variability_configs(
        weather_var_names, weather_var_rev_names, only_default_eval=True, epw_data=epw_data)

weather_variabilities = []#weather_var_config["train_var"] # start with the training config
min_diff = 100000
for row, pca in zip(epw_data.epw_df.iterrows(), epw_data.transformed_df.iterrows()):
    row = row[1]
    pca = pca[1]
    weather_params = {}
    diff = 0
    for variable in weather_var_names + weather_var_rev_names:
        OU_param = np.zeros(3)
        for j in range(3):
            OU_param[j] = np.array(row[f"{variable}_{j}"]).squeeze().item()
        weather_params[variable] = OU_param
        diff += np.mean(np.abs(OU_param))
    weather_variabilities.append(weather_params)
    min_diff = min(diff, min_diff)
print(min_diff)

env_config = {
    # sigma, mean, tau for OU Process
    "weather_variability": weather_variabilities,
    "variability_low": weather_var_config["train_var_low"],
    "variability_high": weather_var_config["train_var_high"],
    "use_rbc": False,
    "use_random": False,
    "sample_environments": True,
    "sinergym_timesteps_per_hour": 1
}


/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


156.5950316905449


In [5]:
from core.uncertain_ppo.uncertain_ppo import UncertainPPOTorchPolicy
env = SinergymWrapper(env_config)
agent = UncertainPPOTorchPolicy.from_checkpoint("checkpoints/vanilla_hot")["default_policy"]

/mnt/7076984276980B4C/ssd_backup/ActiveRL/sinergym_wrappers/sinergym_wrapper.py:333: DeprecationWarning: getName() is deprecated, get the name attribute instead
  self._thread_name = threading.current_thread().getName()


[2023-03-16 20:33:39,481] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-03-16 20:33:39,481] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-03-16 20:33:39,482] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-03-16 20:33:39,482] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-03-16 20:33:39,483] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-03-16 20:33:39,483] EPLUS_ENV_Eplus-5

/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(
/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
env.variables["observation"]

['year',
 'month',
 'day',
 'hour',
 'Site Outdoor Air Drybulb Temperature(Environment)',
 'Site Outdoor Air Relative Humidity(Environment)',
 'Site Wind Speed(Environment)',
 'Site Wind Direction(Environment)',
 'Site Diffuse Solar Radiation Rate per Area(Environment)',
 'Site Direct Solar Radiation Rate per Area(Environment)',
 'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
 'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
 'Zone Air Temperature(SPACE1-1)',
 'Zone Thermal Comfort Mean Radiant Temperature(SPACE1-1 PEOPLE 1)',
 'Zone Air Relative Humidity(SPACE1-1)',
 'Zone Thermal Comfort Clothing Value(SPACE1-1 PEOPLE 1)',
 'Zone Thermal Comfort Fanger Model PPD(SPACE1-1 PEOPLE 1)',
 'Zone People Occupant Count(SPACE1-1)',
 'People Air Temperature(SPACE1-1 PEOPLE 1)',
 'Facility Total HVAC Electricity Demand Rate(Whole Building)']

In [9]:
from captum.attr import IntegratedGradients
import torch
obs = torch.tensor(env.reset(), device="cuda")[None, :]
forward_fn = lambda obs: agent.compute_value_uncertainty(obs)[:, None]
ig = IntegratedGradients(forward_fn)
print(forward_fn(obs))
attributions, delta = ig.attribute(obs, torch.zeros(obs.shape, device="cuda"), target=0, return_convergence_delta=True, n_steps=200)
print('IG Attributions:', attributions[0, 0:20], attributions[0, 20:])
print('Convergence Delta:', delta)

[2023-03-16 20:35:20,730] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-03-16 20:35:20,730] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-03-16 20:35:20,731] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-03-16 20:35:20,731] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-03-16 20:35:20,884] EPLUS_ENV_Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972_MainThread_ROOT INFO:EnergyPlus working directory is in /mnt/7076984276980B4C/ssd_backup/ActiveRL/Eplus-env-Eplus-5Zone-mixed-discrete-stochastic-v1-FlexibleReset70972-res2/Eplus-env-sub_run3
[2023-03-16 20:35:20,884] EPLUS_ENV_Eplus-5Zone-mixed-discrete-s